In [45]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import re
import matplotlib.pyplot as plt
import string
import os
from IPython.display import Image

import pickle as pkl
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [46]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [47]:
from transformers.pytorch_transformers import BertModel, BertTokenizer, BertConfig
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import torch.utils.data as DU

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import json
import base64
# nlp = spacy.load("en_core_web_trf")
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

from utils.tsv_file import TSVFile

In [48]:
from oscar.modeling.modeling_bert import ImageBertForSequenceClassification

oscar_config = BertConfig.from_pretrained('./models/oscar_ir/')

oscar_ir = ImageBertForSequenceClassification.from_pretrained('./models/oscar_ir', config=oscar_config)
oscar_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
torch.manual_seed(123)
random.seed(123)

In [6]:
DATA_DIR = './data/'
IMG_FEAT_DIR = './data/mscoco_imgfeat/'

# DATALOADER

In [ ]:
import pandas as pd 
import cv2
from ast import literal_eval
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
import random
import spacy
from 

class MSMO_Data(Dataset):
    def __init__(self, csv_file, root_dir='/scratch/anshul/valid_data/', transform=None):
        self.transform = transform
        self.listObj = ['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'shoe', 'sports', 'bottle', 'pizza', 'donut', 'chair', 'tv', 'laptop', 'phone', 'blender', 'book', 'clock', 'refrigerator', 'oven', 'microwave', 'oven', 'keyboard']
        self.nlp = spacy.load("en_core_web_trf")
        self.wv = api.load('word2vec-google-news-300')
        self.df = pd.read_csv(root_dir+csv_file)
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        article_file = self.root_dir+'article/'+self.df['article'][idx]
        art = open(article_file)
        art = art.read()
        art = art.replace("\n", " ")
        art = art.rstrip()
        text = art.split("@title ")[1].split("@summary ")[0]
        doc = self.nlp(text)
        ents = [(e.text, e.label_, e.kb_id_) for e in doc.ents]
        text1 = text
        for item in ents:
            if(item[1]=='PERSON'):
                try:
                    vec = wv[item[0]]
                except KeyError:
                    text1=text1.replace(item[0], 'person')
                    continue
                if self.wv.similarity(item[0].split(' ')[0], 'man')>self.wv.similarity(item[0].split(' ')[0], 'woman'):
                    text1=text1.replace(item[0], 'man')
                else:
                    text1=text1.replace(item[0], 'woman')
                
            else:
                maxSim=0.1
                rep = item[0]
                try:
                    vec = wv[item[0]]
                except KeyError:
                    continue
                for obj in self.listObj:
                    if self.wv.similarity(item[0].split(' ')[0], obj) > maxSim:
                        maxSum=self.wv.similarity(item[0].split(' ')[0], obj)
                        rep=obj
                text1=text1.replace(item[0], rep)
        
        #tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
        #text_tokens = tokenizer(text, padding='max_length', max_length=3000)

        
        summaries = art.split("@title ")[1].split("@summary ")[1:]
        summary = ""
        for i in summaries:
            i = i.rstrip()
            summary+=i
            summary+=". "
        
        #summary_tokens = tokenizer(summary, padding='max_length', max_length=300)   

        imgs = []
        x = literal_eval(self.df['img'][idx])
        k=len(x)
        arr = [i for i in range(k)]
        if len(x)>15:
            k=15
            arr = random.sample(range(len(x)), 15)
        for i in arr:
            img_name = self.root_dir+'img/'+x[i]
            im = cv2.imread(img_name)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (150, 150))
            if(self.transform):
                im = self.transform(im)
            imgs.append(im)
        for i in range(15-k):
            imgs.append(np.zeros((150, 150, 3)))
        
        data_pt = {'text':text, 'obj_text':text1, 'summary':summary, 'im_list':imgs}
        return data_pt
        
        



In [ ]:
dataset = MSMO_Data('valid.csv')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [127]:
class OscarInput:
    def __init__(self):
        IMG2IDX_JSON = './img2idx.json'
        
        with open(IMG2IDX_JSON, 'r') as f:
            self.img2idx = json.load(f)
            
        self.features_file = TSVFile(os.path.join(IMG_FEAT_DIR, 'features.tsv'))
        self.labels_file = TSVFile(os.path.join(IMG_FEAT_DIR, 'labels.tsv'))
        
        self.tokenizer = oscar_tokenizer
        self.max_seq_len = 70
        self.max_img_seq_len = 70
        self.att_mask_type = 'CLR'
            
    def make_img2idx(self):
        IMG_FILE = os.path.join(IMG_FEAT_DIR, 'features.tsv')
        LABEL_FILE = os.path.join(IMG_FEAT_DIR, 'labels.tsv')
        img2idx = {}
        with open(IMG_FILE, 'r') as img_file:
            with open(LABEL_FILE, 'r') as label_file:
                cnt = 0
                line = img_file.readline()
                line_label = label_file.readline()
                while line:
                    img_id = line.split('\t')[0].strip()
                    label_id = line.split('\t')[0].strip()

                    if img_id == label_id:
                        img2idx[img_id] = cnt
                        cnt += 1
                    else:
                        print('mismatch... How?')

                    line = img_file.readline()

        with open('./img2idx.json', 'w+') as f:
            json.dump(img2idx, f)

    def get_oscar_input(self, sent, img_id):
        idx = self.img2idx[img_id]
        features_row = self.features_file.seek(idx)
        labels_row = self.labels_file.seek(idx)
        
        # GET LABELS
        results = json.loads(labels_row[1].replace('\'', '\"'))
        objects = results['objects'] if type(
            results) == dict else results
        labels = {
            "image_h": results["image_h"] if type(
                results) == dict else 600,
            "image_w": results["image_w"] if type(
                results) == dict else 800,
            "class": [cur_d['class'] for cur_d in objects],
            "boxes": np.array([cur_d['rect'] for cur_d in objects],
                              dtype=np.float32)
        }
        
        od_labels = ' '.join(labels['class'])
     
        # GET IMAGE FEATURES
        num_boxes = int(features_row[1])
        features = np.frombuffer(base64.b64decode(features_row[-1]),
                                 dtype=np.float32).reshape((num_boxes, -1))
        t_features = torch.from_numpy(features)
        
        # TENSORIZE EXAMPLE
                
        cls_token_segment_id = 0
        pad_token_segment_id = 0
        sequence_a_segment_id = 0
        sequence_b_segment_id = 1
        
        text_a = sent
        text_b = od_labels
        img_feat = t_features
        
        tokens_a = oscar_tokenizer.tokenize(text_a)
        if len(tokens_a) > self.max_seq_len - 2:
            tokens_a = tokens_a[:(self.max_seq_len - 2)]

        tokens = [self.tokenizer.cls_token] + tokens_a + [self.tokenizer.sep_token]
        segment_ids = [cls_token_segment_id] + [sequence_a_segment_id] * (len(tokens_a) + 1)
        seq_a_len = len(tokens)
        if text_b:
            tokens_b = self.tokenizer.tokenize(text_b)
            if len(tokens_b) > self.max_seq_len - len(tokens) - 1:
                tokens_b = tokens_b[: (self.max_seq_len - len(tokens) - 1)]
            tokens += tokens_b + [self.tokenizer.sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

        seq_len = len(tokens)
        seq_padding_len = self.max_seq_len - seq_len
        tokens += [self.tokenizer.pad_token] * seq_padding_len
        segment_ids += [pad_token_segment_id] * seq_padding_len
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # image features
        img_len = img_feat.shape[0]
        if img_len > self.max_img_seq_len:
            img_feat = img_feat[0 : self.max_img_seq_len, :]
            img_len = img_feat.shape[0]
            img_padding_len = 0
        else:
            img_padding_len = self.max_img_seq_len - img_len
            padding_matrix = torch.zeros((img_padding_len, img_feat.shape[1]))
            img_feat = torch.cat((img_feat, padding_matrix), 0)

        # generate attention_mask
        att_mask_type = self.att_mask_type
        if att_mask_type == "CLR":
            attention_mask = [1] * seq_len + [0] * seq_padding_len + \
                             [1] * img_len + [0] * img_padding_len
            
        else:
            # use 2D mask to represent the attention
            max_len = self.max_seq_len + self.max_img_seq_len
            attention_mask = torch.zeros((max_len, max_len), dtype=torch.long)
            # full attention of C-C, L-L, R-R
            c_start, c_end = 0, seq_a_len
            l_start, l_end = seq_a_len, seq_len
            r_start, r_end = self.max_seq_len, self.max_seq_len + img_len
            attention_mask[c_start : c_end, c_start : c_end] = 1
            attention_mask[l_start : l_end, l_start : l_end] = 1
            attention_mask[r_start : r_end, r_start : r_end] = 1
            if att_mask_type == 'CL':
                attention_mask[c_start : c_end, l_start : l_end] = 1
                attention_mask[l_start : l_end, c_start : c_end] = 1
            elif att_mask_type == 'CR':
                attention_mask[c_start : c_end, r_start : r_end] = 1
                attention_mask[r_start : r_end, c_start : c_end] = 1
            elif att_mask_type == 'LR':
                attention_mask[l_start : l_end, r_start : r_end] = 1
                attention_mask[r_start : r_end, l_start : l_end] = 1
            else:
                raise ValueError("Unsupported attention mask type {}".format(att_mask_type))
        
        input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0)
        segment_ids = torch.tensor(segment_ids, dtype=torch.long).unsqueeze(0)
        img_feat = img_feat.unsqueeze(0)
        
        label = 1
        #return idx, tuple([input_ids, attention_mask, segment_ids, img_feat, label])
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': segment_ids,
            'img_feats': img_feat,
            'labels': None
        }
        
    

In [134]:
oscar_inp = OscarInput()
#oscar_inp.make_img2idx()
inp = oscar_inp.get_oscar_input("", "2")
logits = oscar_ir(**inp)[:2][0]
sm = nn.Softmax(dim=1)
probs = sm(logits)
result = probs[:, 1]
result

tensor([5.3426e-06], grad_fn=<SelectBackward>)

In [135]:
dataset[0]['im_list'][0].shape

NameError: name 'dataset' is not defined

In [ ]:
import base64
features = np.frombuffer(base64.b64decode('s'*64),
                         dtype=np.float32).reshape((4, -1))
print(features)
t_features = torch.from_numpy(features)

# OSCAR

# BERTSUM

# MODEL

In [ ]:
class MMSumm(nn.Module):
    def __init__(self, )